In [7]:
!pip install git+https://github.com/mlfoundations/open_clip.git


  Cloning https://github.com/mlfoundations/open_clip.git to /tmp/pip-req-build-9b9g8tbx
  Running command git clone --filter=blob:none --quiet https://github.com/mlfoundations/open_clip.git /tmp/pip-req-build-9b9g8tbx
  Resolved https://github.com/mlfoundations/open_clip.git to commit a87f11eaf354000d2736580855ae0d9b76ad2a22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


**no of samples**

In [8]:
import os

root_dir = "/kaggle/input/pacs-dataset/kfold"
domains = ["photo", "art_painting", "cartoon", "sketch"]

for domain in domains:
    domain_path = os.path.join(root_dir, domain)
    count = 0
    for cls in os.listdir(domain_path):
        cls_path = os.path.join(domain_path, cls)
        count += len(os.listdir(cls_path))
    print(f"{domain} has {count} images")

photo has 1670 images
art_painting has 2048 images
cartoon has 2344 images
sketch has 3929 images


**Multi-Domain Training with RN50 and ViT-B16 Backbones**

In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"
PACS_PATH = "/kaggle/input/pacs-dataset/kfold"

class CLIPMLPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super().__init__()
        self.clip = clip_model
        self.mlp = nn.Sequential(
            nn.Linear(self.clip.visual.output_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        with torch.no_grad():
            feats = self.clip.encode_image(x)
        return self.mlp(feats)

def get_loader(domains, root, transform, batch_size=32, shuffle=False):
    dataset = []
    for domain in domains:
        ds = datasets.ImageFolder(os.path.join(root, domain), transform=transform)
        dataset.extend(ds.samples)
    base_ds = datasets.ImageFolder(os.path.join(root, domains[0]), transform=transform)
    base_ds.samples = dataset
    loader = DataLoader(base_ds, batch_size=batch_size, shuffle=shuffle)
    return loader, len(base_ds.classes)

train_domains = ["photo", "sketch", "cartoon"]
test_domain = "art_painting"
results = {}

backbones = ['RN50', 'ViT-B-16']

for backbone in backbones:
    print(f"\n= Training with {backbone} =")

    clip_model, _, preprocess = open_clip.create_model_and_transforms(backbone, pretrained='openai')
    clip_model.eval().to(device)
    for param in clip_model.parameters():
        param.requires_grad = False

    train_loader, num_classes = get_loader(train_domains, PACS_PATH, preprocess, shuffle=True)
    test_loader, _ = get_loader([test_domain], PACS_PATH, preprocess, shuffle=False)

    model = CLIPMLPClassifier(clip_model, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.mlp.parameters(), lr=1e-4)

    best_acc = 0
    wait = 0
    patience = 10

    for epoch in range(50):
        model.train()
        total_loss = 0

        for imgs, labels in tqdm(train_loader, desc=f"[{backbone}] Epoch {epoch+1} - Training"):
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"[{backbone}] Epoch {epoch+1} - Training Loss: {total_loss:.4f}")

        # Evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in test_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        acc = 100 * correct / total
        print(f"[{backbone}] Test Accuracy on '{test_domain}': {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            wait = 0
            print(f"[{backbone}] Accuracy improved.")
        else:
            wait += 1
            print(f"[{backbone}] No improvement. Wait count: {wait}/{patience}")
            if wait >= patience:
                print(f"[{backbone}] Early stopping triggered.")
                break

    results[backbone] = best_acc
    print(f"[{backbone}] Best Accuracy Achieved: {best_acc:.2f}%")

print("\n= Summary of Test Accuracies =")
for name, acc in results.items():
    print(f"{name:10s}: {acc:.2f}%")



= Training with RN50 =


[RN50] Epoch 1 - Training: 100%|██████████| 249/249 [00:42<00:00,  5.90it/s]


[RN50] Epoch 1 - Training Loss: 437.0409
[RN50] Test Accuracy on 'art_painting': 54.79%
[RN50] Accuracy improved.


[RN50] Epoch 2 - Training: 100%|██████████| 249/249 [00:41<00:00,  6.00it/s]


[RN50] Epoch 2 - Training Loss: 310.9243
[RN50] Test Accuracy on 'art_painting': 54.54%
[RN50] No improvement. Wait count: 1/10


...

[ViT-B-16] Epoch 13 - Training Loss: 16.1565
[ViT-B-16] Test Accuracy on 'art_painting': 96.58%
[ViT-B-16] No improvement. Wait count: 8/10


[ViT-B-16] Epoch 14 - Training: 100%|██████████| 249/249 [01:20<00:00,  3.10it/s]


[ViT-B-16] Epoch 14 - Training Loss: 15.6309
[ViT-B-16] Test Accuracy on 'art_painting': 96.48%
[ViT-B-16] No improvement. Wait count: 9/10


[ViT-B-16] Epoch 15 - Training: 100%|██████████| 249/249 [01:20<00:00,  3.10it/s]


[ViT-B-16] Epoch 15 - Training Loss: 14.8168
[ViT-B-16] Test Accuracy on 'art_painting': 96.24%
[ViT-B-16] No improvement. Wait count: 10/10
[ViT-B-16] Early stopping triggered.
[ViT-B-16] Best Accuracy Achieved: 97.12%

= Summary of Test Accuracies =
RN50      : 62.11%
ViT-B-16  : 97.12%
